### Dutka Krystian 13K1 implementacja klasyfikatora metodą zespołową (Bagging)

In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.datasets import load_wine
from sklearn.datasets import fetch_openml
wine = fetch_openml("wine", version=1, cache=True)
type(wine)

sklearn.utils.Bunch

In [3]:
df = pd.DataFrame(data = np.c_[wine["data"], wine["target"]],
columns = wine["feature_names"] + ['target'])

In [4]:
df.describe

<bound method NDFrame.describe of     Alcohol Malic_acid   Ash Alcalinity_of_ash Magnesium Total_phenols  \
0     14.23       1.71  2.43              15.6     127.0           2.8   
1      13.2       1.78  2.14              11.2     100.0          2.65   
2     13.16       2.36  2.67              18.6     101.0           2.8   
3     14.37       1.95   2.5              16.8     113.0          3.85   
4     13.24       2.59  2.87              21.0     118.0           2.8   
..      ...        ...   ...               ...       ...           ...   
173   13.71       5.65  2.45              20.5      95.0          1.68   
174    13.4       3.91  2.48              23.0     102.0           1.8   
175   13.27       4.28  2.26              20.0     120.0          1.59   
176   13.17       2.59  2.37              20.0     120.0          1.65   
177   14.13        4.1  2.74              24.5      96.0          2.05   

    Flavanoids Nonflavanoid_phenols Proanthocyanins Color_intensity   Hue  \


In [5]:
df.head()

,Alcohol,Malic_acid,Ash,Alcalinity_of_ash,Magnesium,Total_phenols,Flavanoids,Nonflavanoid_phenols,Proanthocyanins,Color_intensity,Hue,OD280%2FOD315_of_diluted_wines,Proline,target
0,14.23,1.71,2.43,15.6,127.0,2.8,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,1
1,13.2,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.4,1050.0,1
2,13.16,2.36,2.67,18.6,101.0,2.8,3.24,0.3,2.81,5.68,1.03,3.17,1185.0,1
3,14.37,1.95,2.5,16.8,113.0,3.85,3.49,0.24,2.18,7.8,0.86,3.45,1480.0,1
4,13.24,2.59,2.87,21.0,118.0,2.8,2.69,0.39,1.82,4.32,1.04,2.93,735.0,1


In [6]:
df.to_csv('wine.csv', header=True)
wine = pd.read_csv('wine.csv')
wine.head()

,Unnamed: 0,Alcohol,Malic_acid,Ash,Alcalinity_of_ash,Magnesium,Total_phenols,Flavanoids,Nonflavanoid_phenols,Proanthocyanins,Color_intensity,Hue,OD280%2FOD315_of_diluted_wines,Proline,target
0,0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,1
1,1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,1
2,2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,1
3,3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,1
4,4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,1


In [7]:
wine = wine.drop('Unnamed: 0', axis = 1)
wine.head()

,Alcohol,Malic_acid,Ash,Alcalinity_of_ash,Magnesium,Total_phenols,Flavanoids,Nonflavanoid_phenols,Proanthocyanins,Color_intensity,Hue,OD280%2FOD315_of_diluted_wines,Proline,target
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,1
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,1
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,1
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,1
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,1


In [8]:
wine.columns

Index(['Alcohol', 'Malic_acid', 'Ash', 'Alcalinity_of_ash', 'Magnesium',
       'Total_phenols', 'Flavanoids', 'Nonflavanoid_phenols',
       'Proanthocyanins', 'Color_intensity', 'Hue',
       'OD280%2FOD315_of_diluted_wines', 'Proline', 'target'],
      dtype='object')

In [9]:
def bootstrap(dataset, bootstrap_count, sample_count):
    bootstrap_list=[]
    taken_indexes=set()
    for i in range(bootstrap_count):
        bootstrap_index=list(np.random.randint(0, len(dataset), size=sample_count))
        taken_indexes=taken_indexes|set(bootstrap_index)
        bootstrap_list.append( dataset.iloc[bootstrap_index])
    oob_set=dataset.iloc[list(set(range(len(df)))-taken_indexes)]
    return bootstrap_list, oob_set

In [10]:
bs, oob = bootstrap(df,3,5)

In [11]:
oob

,Alcohol,Malic_acid,Ash,Alcalinity_of_ash,Magnesium,Total_phenols,Flavanoids,Nonflavanoid_phenols,Proanthocyanins,Color_intensity,Hue,OD280%2FOD315_of_diluted_wines,Proline,target
0,14.23,1.71,2.43,15.6,127.0,2.8,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,1
1,13.2,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.4,1050.0,1
4,13.24,2.59,2.87,21.0,118.0,2.8,2.69,0.39,1.82,4.32,1.04,2.93,735.0,1
6,14.39,1.87,2.45,14.6,96.0,2.5,2.52,0.3,1.98,5.25,1.02,3.58,1290.0,1
7,14.06,2.15,2.61,17.6,121.0,2.6,2.51,0.31,1.25,5.05,1.06,3.58,1295.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,14.16,2.51,2.48,20.0,91.0,1.68,0.7,0.44,1.24,9.7,0.62,1.71,660.0,3
173,13.71,5.65,2.45,20.5,95.0,1.68,0.61,0.52,1.06,7.7,0.64,1.74,740.0,3
174,13.4,3.91,2.48,23.0,102.0,1.8,0.75,0.43,1.41,7.3,0.7,1.56,750.0,3
176,13.17,2.59,2.37,20.0,120.0,1.65,0.68,0.53,1.46,9.3,0.6,1.62,840.0,3


## Training
### DecisionTreeClassifier, SGDClassifier, RandomForestClassifier

In [12]:
from sklearn.model_selection import train_test_split
from statistics import mode
from sklearn.metrics import accuracy_score

In [13]:
X_g, y_g = df.iloc[:, :-1].values, df.iloc[:, -1].values

In [14]:
np.random.randint(0, 70, size=10)

array([50, 26, 32, 34,  2, 67, 15, 38, 11, 31])

In [15]:
X_train_g, X_test_g, y_train_g, y_test_g = train_test_split(X_g, y_g, test_size = 0.2)

In [16]:
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

c1 = SVC()
c2 = DecisionTreeClassifier(max_depth = 4)
c3 = KNeighborsClassifier(n_neighbors=5)
list_of_models = [c1,c2,c3]

In [17]:
bs, oob = bootstrap(df,3,40)
j=0
list_of_results = [('svc', c1), ('tree', c2), ('kneighbors', c3)]
for i in list_of_models:
    data=bs[j]
    X_train, y_train = data.iloc[:, :-1].values, data.iloc[:, -1].values
    X_test, y_test= oob.iloc[:, :-1].values, oob.iloc[:, -1].values
    print(i.fit(X_train, y_train))
    print(i.score(X_train, y_train), i.score(X_test, y_test))
    #print(i.predict(X_train))
    j=j+1

SVC()
0.725 0.5952380952380952
DecisionTreeClassifier(max_depth=4)
1.0 0.9404761904761905
KNeighborsClassifier()
0.8 0.7142857142857143


In [18]:
#vch = VotingClassifier(list_of_results, voting = 'hard', verbose = True)
#vch.fit(X_train_g, y_train_g)
#vch.score(X_train_g, y_train_g), vch.score(X_test_g, y_test_g)

In [20]:
scores_train = []
scores_test = []
for i in list_of_models:
    score = list(i.predict(X_train_g))
    scores_train.append(score)
    score_t = list(i.predict(X_test_g))
    scores_test.append(score_t)
    
scores_train=np.array(scores_train)
scores_test=np.array(scores_test)

#print(np.array(scores_train).shape)

hv_predicted_train = [mode(v) for v in np.array(scores_train).T]
hv_predicted_test = [mode(v) for v in np.array(scores_test).T]

hv_accuracy_train=accuracy_score(y_train_g, hv_predicted_train)
hv_accuracy_test=accuracy_score(y_test_g, hv_predicted_test)
print("Wynik dla danych treningowych (hard voting):")
print(hv_accuracy_train)
print("Wynik dla danych testowych (hard voting):")
print(hv_accuracy_test)


Wynik dla danych treningowych (hard voting):
0.7816901408450704
Wynik dla danych testowych (hard voting):
0.8611111111111112
